In [57]:
import time
import numpy as np
from itertools import combinations_with_replacement
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
from itertools import combinations_with_replacement
from sympy.core import Mul
import copy

In [58]:
data = pd.read_csv('Taxinfo.csv')

In [59]:
data

,Unnamed: 0,HHI,HHDL,Married,CollegGrads,AHHAge,Cars,Filed in 2017,Filed in 2016,Filed in 2015,PoliticalParty
0,1,49685,227187,0,0,105,0,1,1,1,Democrat
1,2,64756,-507342,2,3,68,3,1,0,0,Independent
2,3,115435,521290,1,3,81,2,0,1,0,Republican
3,4,99454,251829,2,1,52,4,1,0,0,Republican
4,5,157274,-472337,0,1,28,1,1,0,1,Independent
...,...,...,...,...,...,...,...,...,...,...,...
999,1000,37626,20830,0,1,38,4,0,1,0,Democrat
1000,1001,79765,63483,2,1,71,5,0,1,0,Republican
1001,1002,111219,-573449,0,4,35,2,0,0,0,Republican
1002,1003,105591,-611207,0,1,59,1,1,1,0,Republican


In [60]:
data.drop(data.columns[[ 0]], axis=1, inplace=True)
    
    
    # convert categorical labels to numbers
diag_map = {'Democrat': 1.0, 'Independent': -1.0,'Republican': -1.0}
data['PoliticalParty'] = data['PoliticalParty'].map(diag_map)
    

In [61]:
data

,HHI,HHDL,Married,CollegGrads,AHHAge,Cars,Filed in 2017,Filed in 2016,Filed in 2015,PoliticalParty
0,49685,227187,0,0,105,0,1,1,1,1.0
1,64756,-507342,2,3,68,3,1,0,0,-1.0
2,115435,521290,1,3,81,2,0,1,0,-1.0
3,99454,251829,2,1,52,4,1,0,0,-1.0
4,157274,-472337,0,1,28,1,1,0,1,-1.0
...,...,...,...,...,...,...,...,...,...,...
999,37626,20830,0,1,38,4,0,1,0,1.0
1000,79765,63483,2,1,71,5,0,1,0,-1.0
1001,111219,-573449,0,4,35,2,0,0,0,-1.0
1002,105591,-611207,0,1,59,1,1,1,0,-1.0


In [62]:
Y = data.loc[:, 'PoliticalParty']
X = data.iloc[:, 0:-1]

In [63]:
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size = 0.4, random_state = 42)

In [64]:
def Multiply(a):
    ans = 1
    for i in a:
        ans*=i
    return ans

In [65]:
def Create_features(variables):
    ans = []
    for i in range(1,5):
        l = list(combinations_with_replacement(variables, i))
        for j in l:
            ans.append(Multiply(j))
            #print(Multiply(j))
    ans.append(1)
    ans = np.array(ans).astype(float)
    return ans.reshape((ans.shape[0],1)) 

In [66]:
def smo(X,Y, C=0.1, toler=0.001, maxIter=5):
    #dataMatrix = mat(dataMatIn); 
    #labelMat = mat(classLabels).transpose()
    b = 0; m = X.shape[0]
    alpha=np.zeros(shape=(X.shape[0],1))
    #alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): #Fetch one piece of data each time, corresponding to alpha1
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b #Calculate the classification function result, directly apply the classification function formula (8)
            Ei = fXi-float(labelMat[i])#if checks if an example violates KKT conditions #The difference between the calculated result and the actual classification
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                               #3 cabinets that do not meet the KKT condition in the iteration are used here, ((labelMat[i]*Ei> toler) and (alphas[i]> 0) as an example, assuming this is a positive classification, then (labelMat[i ]*Ei> toler)
                               # Ei>0 and Ei> toler, because labelMat[i]=1, yi*fXi>1 is obtained, at this time, alpha1>0 is not satisfied, and the original alpha1=0
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b # alpha2 random selection
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]): # y1 and y1 have different signs, calculate L and H
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: 
                    print ("L==H") 
                    continue
                    eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:]. T #Calculate η
                if eta >= 0: 
                    print ("eta>=0") 
                    continue #When calculating η =0, divide by 0, skip this case temporarily
                    alphas[j] -= labelMat[j]*(Ei-Ej)/eta #Calculate alpha2, refer to formula (5)
                    alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j]-alphaJold) <0.00001):
                    print ("j not moving enough")
                    continue #reach the optimization threshold
                    alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                                                                                                 #the update is in the oppostie direction Update alpha1, formula (6)
                                 #Calculate b1,b2, formula (7)                                                        
                    b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                    b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                    if (0 < alphas[i]) and (C > alphas[i]):
                        b = b1
                    elif (0 < alphas[j]) and (C > alphas[j]): 
                        b = b2
                    else: 
                        b = (b1 + b2)/2.0
                        alphaPairsChanged += 1
                        print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
                if (alphaPairsChanged == 0): 
                    iter += 1
                else: 
                    iter = 0
        print ("iteration number: %d" % iter)
         
        return b,alphas
    

In [67]:
def predict(X,Y,alpha,b,x,sigma):
    result=0.0

    for i in range(X.shape[0]):
        result+=(alpha[i]*Y[i]*(X[i,:] , x,sigma));

    result+=b

    return result

In [68]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)
    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [69]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [70]:
def computeW(alphas, X, Y):
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*Y[i],X[i,:].T)
        return w

In [71]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    #if only one example is passed (eg. in case of SGD)
    if type(Y_batch) == np.float64:
        Y_batch = np.array([Y_batch])
        X_batch = np.array([X_batch])  # gives multidimensional array

    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))

    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (regularization_strength * Y_batch[ind] * X_batch[ind])
        dw += di

    dw = dw/len(Y_batch)  # average
    return dw

In [72]:
def sgd(features, outputs):
    max_epochs = 500
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [73]:
def init1():
    print("reading dataset...")
    # read data in pandas (pd) data frame
    data = pd.read_csv('Taxinfo.csv')
    
    data.drop(data.columns[[ 0]], axis=1, inplace=True)
    
    
    # convert categorical labels to numbers
    diag_map = {'Democrat': 1.0, 'Independent': -1.0,'Republican': -1.0}
    data['PoliticalParty'] = data['PoliticalParty'].map(diag_map)
    
    Y = data.loc[:, 'PoliticalParty']
    X = data.iloc[:, 0:-1]
    
    
    # filter features
    #remove_correlated_features(X)
    Xnew = np.apply_along_axis(Create_features, 1, X)
    #X = pd.DataFrame(Xnew)
    #remove_less_significant_features(X, Y)
    # normalize data for better convergence and to prevent overflow
    #X_normalized = MinMaxScaler().fit_transform(Xnew)
    #X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    #X.insert(loc=len(X.columns), column='intercept', value=1)
    # split data into train and test set
    print("splitting dataset into train and test sets...")
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
    # train the model
    print("training started...")
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    #w = computeW(alphas,X,Y)
    print("training finished.")
    print("weights are: {}".format(W))
    # testing the model
    print("testing the model...")
    y_train_predicted = np.array([])
    for i in range(X_train.shape[0]):
        yp = np.sign(np.dot(X_train.to_numpy()[i], W))
        #y_train_predicted= predict(y_train_predicted, yp)
        y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)
        #y_test_predicted= predict(y_test_predicted, yp)
    # An unknow data given by me to check which class it belongs to
    #a = np.array([0.8, 0.6, 0.75, 0.75, 1])
    #yp = np.sign(np.dot(a, W))
    #print("Y value for a : ",yp)
    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    
regularization_strength = 10000
learning_rate = 0.000001


# In[14]:


# init1 assure if a data point belongs to class 1 or not #
# For Class 1 #

start = time.time()
init1()
end = time.time()
print("TIME TAKEN: {} \n".format(end-start))


reading dataset...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in longlong_scalars
  after removing the cwd from sys.path.


splitting dataset into train and test sets...
training started...
Epoch is: 1 and Cost is: 5645155403654.373
Epoch is: 2 and Cost is: 7837671428034.185
Epoch is: 4 and Cost is: 3308618113157.494
Epoch is: 8 and Cost is: 6558909953936.499
Epoch is: 16 and Cost is: 3478994787682.0854
Epoch is: 32 and Cost is: 8949103793585.324
Epoch is: 64 and Cost is: 417290248770.3373
Epoch is: 128 and Cost is: 7693265858905.352
Epoch is: 256 and Cost is: 7398652363212.388
Epoch is: 499 and Cost is: 6515285132608.896
training finished.
weights are: [-5232.83182655 -2839.09831233   -96.16910922  -162.12006909
 -2841.59658798  -204.05012048   -94.70579495   -10.81359511
   -94.16268024]
testing the model...
accuracy on test dataset: 0.5373134328358209
recall on test dataset: 0.3972602739726027
precision on test dataset: 0.3972602739726027
TIME TAKEN: 8.696686506271362 

